# VAEと拡散モデル

VAEと拡散モデルでは、未来予測を使った意思決定を、実装可能な形に分解して学びます。


## 実験 1: VAE/拡散併用の遷移初期化

世界モデルでの生成的遷移を意識し、遷移係数を設定します。


In [ ]:
z_t = 0.27
a_t = -0.6
A, B = 0.94, 0.14
z_next = A * z_t + B * a_t
print('task = vae-diffusion-world-models')
print('z_next =', round(z_next, 6))

この遷移に生成モデルを重ねる観点を後続で扱います。

この節では、状態表現 が入出力のどこを決めるかを中心に読める状態になれば十分です。


## 実験 2: 観測予測を作る

次に、潜在状態から観測を復元する写像を作ります。状態推定と観測再現の役割分担をコードで掴みます。


In [ ]:
def decode(z):
    return {'position': 2.5 * z + 0.1, 'velocity': 0.8 * z - 0.05}
obs_next = decode(z_next)
print('obs_next =', {k: round(v, 4) for k, v in obs_next.items()})
print('keys =', list(obs_next.keys()))

観測予測を別関数に切ると、遷移誤差と観測誤差を分離して調整できます。

この節では、状態表現 が入出力のどこを決めるかを中心に読める状態になれば十分です。


## 式と実装の往復

1. $z_{t+1} = f_{\theta}(z_t, a_t)$
2. $\hat{o}_{t+1} = g_{\theta}(z_{t+1})$


## 実験 3: ロールアウトを試す

ここで複数ステップ予測を実行します。1ステップでは見えない誤差累積を把握するためです。


In [ ]:
actions = [0.0, 1.0, 1.0, 0.0, -0.5]
z = 0.1
traj = []
for a in actions:
    z = 0.92 * z + 0.18 * a
    traj.append(round(z, 5))
print('rollout =', traj)

長期予測で崩れるなら、遷移モデルの安定性や状態表現の情報量不足を疑います。

この節では、状態表現 が入出力のどこを決めるかを中心に読める状態になれば十分です。


## 実験 4: 計画候補を比較する

次に、複数の行動列を比較して、どの計画が望ましいかを評価します。モデルベース強化学習の中心操作です。


In [ ]:
plans = [[0, 1, 1], [1, 1, 1], [0, 0, 1]]
def score_plan(plan):
    z = 0.1
    for a in plan:
        z = 0.92 * z + 0.18 * a
    return z
scores = [round(score_plan(p), 5) for p in plans]
print('scores =', scores)

計画評価が可能になると、実環境での試行回数を抑えた探索がしやすくなります。

この節では、状態表現 が入出力のどこを決めるかを中心に読める状態になれば十分です。


## 実験 5: モデル誤差を監視する

最後に、予測と実測の差を定量化します。世界モデルは『予測できる範囲』を常に点検する運用が重要です。


In [ ]:
pred = [0.10, 0.22, 0.31, 0.29]
real = [0.11, 0.25, 0.28, 0.35]
errors = [abs(p - r) for p, r in zip(pred, real)]
print('errors =', [round(e, 4) for e in errors])
print('mean_error =', round(sum(errors) / len(errors), 5))

平均誤差だけでなく時点別誤差を追うと、どの遷移条件でモデルが弱いかを特定しやすくなります。

この節では、状態表現 が入出力のどこを決めるかを中心に読める状態になれば十分です。


## 振り返り

今回のノートで押さえておくべき誤解しやすい点を整理します。

1. 1ステップ誤差だけで安心してしまう
2. 長期予測の誤差爆発を監視しない
3. 状態表現が不足しているのにモデルだけ調整する
